In [2]:
import numpy as np
import pandas as pd
import scipy.sparse as sps
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
import pickle
import os.path

from collections import defaultdict

# I. Класс реализованного метода

In [12]:
class UserBasedRecommender():
    def __init__(self, path_to_orders, path_to_order_products_train, decomposition_dim):
        '''
        :param path_to_orders: путь к исходному датафрейму с keggle orders.csv
        :param path_to_order_products_train: путь к новому тренировочному датафрейму, полученному в п.II ниже
        :param decomposition_dim: размерность декомпозиции
        '''
        self.path_to_orders = path_to_orders
        self.path_to_order_products_train = path_to_order_products_train
        self.n_comp = decomposition_dim
        
        self.ok_status = True
        check, message = self.check_input_params()
        
        if not check:
            self.ok_status = False
            print(message)
    
    def check_input_params(self):
        if not os.path.isfile(self.path_to_orders):
            return False, 'File doesnt exist'
        if not os.path.isfile(self.path_to_order_products_train):
            return False, 'File doesnt exist'
        if not isinstance(self.n_comp, int) or not self.n_comp > 0:
            return False, 'decomposition_dim should be positive integer value'
        
        return True, 'Ok'
            
    def read_data(self):
        if not self.ok_status:
            return 'There were some problems'
        
        self.orders = pd.read_csv(self.path_to_orders)
        self.order_products_train = pd.read_csv(self.path_to_order_products_train)
    
    def prepare_data(self):
        '''
        создает таблицу user-item для дальнейших манипуляций
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        self.merged = pd.merge(self.orders, self.order_products_train, on='order_id', how='right')
        self.merged = self.merged[['order_id', 'user_id', 'product_id']]
        
        self.user_id_num = {key: value for value, key in enumerate(self.merged['user_id'].unique(), start=0)}
        self.product_id_num = {key: value for value, key in enumerate(self.merged['product_id'].unique(), start=0)}
        self.product_num_id = {value: key for value, key in enumerate(self.merged['product_id'].unique(), start=0)}
        
        self.merged['user_num'] = self.merged['user_id'].transform(lambda x: self.user_id_num[x])
        self.merged['product_num'] = self.merged['product_id'].transform(lambda x: self.product_id_num[x])
        self.merged['buy'] = 1
        
        self.user_item_matrix = sps.coo_matrix((self.merged.buy, (self.merged.user_num, self.merged.product_num)))
        
    def decompose(self):
        '''
        декомпозиция user-item_matrix с размерностью n_comp
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        self.model = NMF(n_components=self.n_comp, init='random', random_state=0)
        self.user_matrix = self.model.fit_transform(self.user_item_matrix)
        self.product_matrix = self.model.components_
    
    def get_similarity(self, top_n_size):
        '''Каждому юзеру сопоставляет id n-схожих юзеров
        :param top_n_size: сколько схожих пользователей по убыванию схожести надо сохранить
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        self.similar_users_df = pd.DataFrame(columns=[i for i in range(1, top_n_size + 1)])

        for user_num in tqdm(range(len(self.user_matrix))):
          user_to_others_similarity = cosine_similarity(X=self.user_matrix, Y=self.user_matrix[user_num].reshape(1, self.n_comp))
          top_n = pd.DataFrame(user_to_others_similarity).sort_values(by=0, ascending = False)[:top_n_size].index
          self.similar_users_df.loc[user_num] = top_n
    
    def save_similarity_df(self, path):
        if not self.ok_status:
            return 'There were some problems'
        
        self.similar_users_df.to_csv(path, index=False)
        
    def set_similarity_df(self, path_to_df):
        '''
        Если уже есть посчитанная матрица, то можем использовать и обойти долгий кусок обучения в fit(),
        указав параметр loaded_df=True
        :param path_to_df: путь до csv файла с учебным датафреймом
        '''
        if not os.path.isfile(path_to_df):
            # оформить ошибки по-человечески - нужен же возврат к True
            # self.ok_status = False
            return 'File doesnt exist'
        
        self.similar_users_df = pd.read_csv(path_to_df)
    
    def get_predicted_user_item_matrix(self):
        pass
    
    def fit(self, path_to_save=False, loaded_df=False, top_n_size=5):
        '''
        :param self:
        :param path_to_save: путь для сохранения матрицы схожести
        :param loaded_df: bool - True если матрица уже была загружена и пересчитывать заново ее не нужно;
                          default=False
        :param top_n_size: размер топ схожих пользователей 
        :return: информация в случае ошибки 
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        self.top_n_size = top_n_size
        
        self.read_data()     # читает необходимые df
        self.prepare_data()  # создание таблицы user-item с необх. полями и разметками
        self.decompose()     # декомпозиция матрицы на 2 матрицы размерности n
        if not loaded_df:
            self.get_similarity(self.top_n_size)
            if path_to_save:
                self.save_similarity_df(path_to_save)
        
    def predict(self, user_id, sim_users_count=False):
        '''
        :param self:
        :param user_id:  id юзера, для которого считаем предсказание покупок
        :param sim_users_count: если нужно учесть не весь топ схожих юзеров, а только его часть, 
                                равную значению данного параметра
        :return: сообщение в случае ошибки; в случае успеха - словарь {product_id: сумма весов}
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        if user_id in self.user_id_num:
            user_num = self.get_user_num(user_id)
        else:
            return 'There is no user with this id'
        
        if not sim_users_count or sim_users_count > self.top_n_size:
            sim_users_count = self.top_n_size
        
        users_count = 0
        
        # индексы n_comp схожих пользователей
        sim_users_nums = self.similar_users_df.loc[user_num].values
        d = defaultdict(int)
        for sim_user_num in sim_users_nums:
            # df в форме: user_num | product_num | buy - поэтому по index получаем номер продукта в таблице
            sim_user_purchases = pd.Series.sparse.from_coo(self.user_item_matrix)[sim_user_num].index
            sim_user_weight = cosine_similarity(X=self.user_matrix[sim_user_num].reshape(1, self.n_comp), Y=self.user_matrix[user_num].reshape(1, self.n_comp))
            
            for product in sim_user_purchases:
                product_id = self.product_num_id[product]
                d[product_id] += sim_user_weight
            
            users_count += 1 
            if users_count == sim_users_count:
                break
        
        return dict(sorted(d.items(), key=lambda x: x[1], reverse=True))
    
    def get_coo(self):
        return self.user_item_matrix
    
    def get_user_num(self, user_id):
        return self.user_id_num[user_id]
    
    def prepare_validation_data(self, path_to_valid_df):
        '''
        создает таблицу user-item для дальнейших манипуляций
        :param path_to_valid_df: путь до csv таблицы с валидационным датафреймом
        '''
        if not self.ok_status:
            return 'There were some problems'
        
        self.order_products_valid = pd.read_csv(path_to_valid_df)
        
        self.merged_valid = pd.merge(self.orders, self.order_products_valid, on='order_id', how='right')
        self.merged_valid = self.merged_valid[['order_id', 'user_id', 'product_id']]
        
        self.user_id_num_valid = {key: value for value, key in enumerate(self.merged_valid['user_id'].unique(), start=0)}
        self.product_id_num_valid = {key: value for value, key in enumerate(self.merged_valid['product_id'].unique(), start=0)}
        self.product_num_id_valid = {value: key for value, key in enumerate(self.merged_valid['product_id'].unique(), start=0)}
        
        self.merged_valid['user_num'] = self.merged_valid['user_id'].transform(lambda x: self.user_id_num_valid[x])
        self.merged_valid['product_num'] = self.merged_valid['product_id'].transform(lambda x: self.product_id_num_valid[x])
        self.merged_valid['buy'] = 1
        
        self.user_item_matrix_valid = sps.coo_matrix((self.merged_valid.buy, (self.merged_valid.user_num, self.merged_valid.product_num)))
        
    def check_metrics(self, count_users=False, sim_users_count=False):
        '''
        :param count_users: для скольких юзеров посчитаем метрики
        :param sim_users_count: сколько юзеров из топа схожих будут учитываться для предсказания покупок
        '''
        predicted_purchases = {}
        actual_purchases = {}
        
        count = 0
        
        for user_id in tqdm(self.user_id_num):
            predicted = [*self.predict(user_id, sim_users_count=sim_users_count)]
            
            if user_id not in self.user_id_num_valid:
                print('No user with id: ', user_id)
                continue
            user_num_in_valid_set = self.user_id_num_valid[user_id]
            user_actual_purchases_nums = pd.Series.sparse.from_coo(self.user_item_matrix_valid)[user_num_in_valid_set].index
            actual = []
            for product_num in user_actual_purchases_nums:
                product_id = self.product_num_id_valid[product_num]
                actual.append(product_id)
                
            predicted_purchases[user_id] = predicted
            actual_purchases[user_id] = actual
            
            count +=1
            if count_users and count == count_users:
                break
        
        return actual_purchases, predicted_purchases
        # self.result = mapk(actual_purchases.values(), predicted_purchases.values(), k=10)
        
        # return self.result
        
            
        

## Пример работы с обучением

In [ ]:
# файл, куда по необходимости загрузим результаты
path_to_res = 'data/results.csv'

# необходимые файлы: 
# 1. исходный из keggle orders
path_to_orders = 'data/orders.csv'

# 2. созданный-смерженный как надо методами из п.II
df_train_path = 'data_split/train_data.csv'

# размерность для декомпозиции
n_comp = 10

# top_n
top_n_size = 5

# создаем экземпляр класса с вышеописанными параметрами
Recommender = UserBasedRecommender(path_to_orders, df_train_path, n_comp)

# Обучение с параметром path_to_save=filename после рассчета матрицы схожести сохранит df по указанному пути
# обучение длится долго, так что крайне советую этот параметр указать
Recommender.fit(path_to_save=path_to_res, top_n_size=top_n_size)

# можем пресказать что-то для юзера с id=12345 (пример выводы в следующей ячейке, так как матрица у меня уже была посчитана)
Recommender.predict(125)

# результат - номер продукта и пока просто сумма весов (те никак не нормализованное число, посчитано по матрице с 0/1)

### Считаем датасет побольше

In [40]:
# файл, куда по необходимости загрузим результаты
path_to_res = 'data/results_top_100_dim_50.csv'

# необходимые файлы: 
# 1. исходный из keggle orders
path_to_orders = 'data/orders.csv'

# 2. созданный-смерженный как надо методами из п.II
df_train_path = 'data_split/train_data.csv'

# размерность для декомпозиции
n_comp = 50

# top_n
top_n_size = 100

# создаем экземпляр класса с вышеописанными параметрами
Recommender = UserBasedRecommender(path_to_orders, df_train_path, n_comp)

# Обучение с параметром path_to_save=filename после рассчета матрицы схожести сохранит df по указанному пути
# обучение длится долго, так что крайне советую этот параметр указать
Recommender.fit(path_to_save=path_to_res, top_n_size=top_n_size)

# можем пресказать что-то для юзера с id=12345 (пример выводы в следующей ячейке, так как матрица у меня уже была посчитана)
# Recommender.predict(125)

# результат - номер продукта и пока просто сумма весов (те никак не нормализованное число, посчитано по матрице с 0/1)

D:\Soft\Anaconda\lib\site-packages\ipykernel_launcher.py:70: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [41]:
Recommender.predict(15)

[(12427, array([[42.9785402]])),
 (31651, array([[32.98164413]])),
 (37710, array([[32.98155293]])),
 (39657, array([[18.9889966]])),
 (21572, array([[17.98924481]])),
 (32455, array([[16.99004398]])),
 (47402, array([[14.99245467]])),
 (41400, array([[13.99215955]])),
 (3298, array([[11.9930688]])),
 (35561, array([[10.99479269]])),
 (11759, array([[10.99421759]])),
 (13575, array([[8.99495154]])),
 (38928, array([[8.99476011]])),
 (45051, array([[7.99549273]])),
 (6184, array([[7.99528558]])),
 (32478, array([[7.99525518]])),
 (40199, array([[6.9963011]])),
 (18023, array([[6.99623119]])),
 (11266, array([[6.99609757]])),
 (46521, array([[6.99568192]])),
 (22362, array([[5.99649672]])),
 (16732, array([[5.99639352]])),
 (13042, array([[4.99715293]])),
 (16974, array([[4.99713299]])),
 (26348, array([[4.99712198]])),
 (6729, array([[4.99708671]])),
 (130, array([[3.99825302]])),
 (8013, array([[3.99805701]])),
 (30591, array([[3.99781087]])),
 (4938, array([[3.99767599]])),
 (10441, a

## Пример работы без обучения (то есть уже есть просчитанная матрица - загрузим ее и сделаем предсказание):

In [149]:
# файл, где лежат результаты
path_to_res = 'data/results.csv'

# необходимые файлы: 
# 1. исходный из keggle orders
path_to_orders = 'data/orders.csv'

# 2. созданный-смерженный как надо методами из п.II
df_train_path = 'data_split/train_data.csv'

# размерность для декомпозиции
n_comp = 10

# создаем экземпляр класса с вышеописанными параметрами
Recommender = UserBasedRecommender(path_to_orders, df_train_path, n_comp)

# Загружаем уже посчитанные результаты
Recommender.set_similarity_df(path_to_res)

# Обучение включает с параметром loaded_df=True создаст только необходимые для пресказания матрицы, 
# не относящиеся к уже имеющимся результатам (по дефолту False и для крохотных размерностей считалось 3 часа...)
Recommender.fit(loaded_df=True)

# можем пресказать что-то для юзера с id=125
Recommender.predict(125)

# результат - номер продукта и пока просто сумма весов (те никак не нормализованное число, посчитано по матрице с 0/1)

[(29487, array([[4.99052653]])),
 (19348, array([[2.99291994]])),
 (39646, array([[2.]])),
 (11827, array([[1.]])),
 (27676, array([[1.]])),
 (31564, array([[1.]])),
 (47502, array([[1.]])),
 (42284, array([[1.]])),
 (12620, array([[1.]])),
 (5460, array([[1.]])),
 (13866, array([[1.]])),
 (26965, array([[1.]])),
 (7877, array([[1.]])),
 (43056, array([[1.]])),
 (10613, array([[1.]])),
 (9755, array([[1.]])),
 (34726, array([[1.]])),
 (37543, array([[1.]])),
 (13287, array([[1.]])),
 (29176, array([[1.]])),
 (20463, array([[1.]])),
 (49079, array([[0.99796744]])),
 (38279, array([[0.99796744]])),
 (12911, array([[0.99796744]])),
 (18681, array([[0.99796744]])),
 (39282, array([[0.99796744]])),
 (48927, array([[0.99761497]])),
 (9387, array([[0.99760658]])),
 (41591, array([[0.99760658]])),
 (7736, array([[0.99760658]])),
 (432, array([[0.99760658]])),
 (13921, array([[0.99760658]])),
 (45835, array([[0.99760658]])),
 (39411, array([[0.99760658]])),
 (42441, array([[0.99760658]])),
 (27

### Без обучения датасет побольше

In [43]:
# файл, где лежат результаты
path_to_res = 'data/results_top_100_dim_50.csv'

# необходимые файлы: 
# 1. исходный из keggle orders
path_to_orders = 'data/orders.csv'

# 2. созданный-смерженный как надо методами из п.II
df_train_path = 'data_split/train_data.csv'

# размерность для декомпозиции
n_comp = 50

# создаем экземпляр класса с вышеописанными параметрами
Recommender_1 = UserBasedRecommender(path_to_orders, df_train_path, n_comp)

# Загружаем уже посчитанные результаты
Recommender_1.set_similarity_df(path_to_res)

# Обучение включает с параметром loaded_df=True создаст только необходимые для пресказания матрицы, 
# не относящиеся к уже имеющимся результатам (по дефолту False и для крохотных размерностей считалось 3 часа...)
Recommender_1.fit(loaded_df=True)

# можем пресказать что-то для юзера с id=125
Recommender_1.predict(125)

# результат - номер продукта и пока просто сумма весов (те никак не нормализованное число, посчитано по матрице с 0/1)

[(29487, array([[91.6138716]])),
 (44799, array([[10.72094683]])),
 (23909, array([[9.69193343]])),
 (5025, array([[8.59713041]])),
 (19348, array([[5.96111546]])),
 (38200, array([[4.92975644]])),
 (15424, array([[3.95047321]])),
 (7877, array([[3.94330576]])),
 (27796, array([[3.83918282]])),
 (4210, array([[3.7334297]])),
 (7751, array([[3.71143672]])),
 (31981, array([[3.65704023]])),
 (33129, array([[2.98370453]])),
 (24122, array([[2.98356623]])),
 (45531, array([[2.97168248]])),
 (25830, array([[2.96994756]])),
 (49261, array([[2.96994756]])),
 (34024, array([[2.96803505]])),
 (48205, array([[2.96369141]])),
 (44570, array([[2.9603839]])),
 (15923, array([[2.95986943]])),
 (11461, array([[2.95714243]])),
 (5818, array([[2.9450172]])),
 (31683, array([[2.94104105]])),
 (19863, array([[2.93820977]])),
 (27710, array([[2.93556558]])),
 (11265, array([[2.90922404]])),
 (23888, array([[2.89045255]])),
 (8195, array([[2.87101285]])),
 (16953, array([[2.8082905]])),
 (15359, array([[2.

# II. Пропишем все пути и сконструируем датасеты мечты

In [99]:
import pickle

# необходимые файлы с исходными датасетами с keggle
path_data_train = 'D:/Projects/Anaconda/EDA/data/order_products__train.csv'
path_data_prior = 'D:/Projects/Anaconda/EDA/data/order_products__prior.csv'
path_data_orders = 'D:/Projects/Anaconda/EDA/data/orders.csv'
    
# [куда запишем]/[где уже лежат] файлы с индексами
train_path = 'data_split/train_indices.pickle'
valid_path = 'data_split/validation_indices.pickle'
test_path = 'data_split/test_indices.pickle'

# куда запишем файлы с новым разделением
df_train_path = 'data_split/train_data.csv'
df_valid_path = 'data_split/validation_data.csv'
df_test_path = 'data_split/test_data.csv'

### Если файлов с индексами нет

In [102]:
def _split_indices(grouped_ratings, retriever):
    return np.concatenate(grouped_ratings.apply(retriever).values)

def split(path_to_orders):
    orders = pd.read_csv(path_to_orders)
    grouper = orders.sort_values('order_number').groupby('user_id')
    train_indices = _split_indices(
        grouper,
        lambda user_ratings: user_ratings[:int(user_ratings.shape[0] * 0.5)].index.values)
    
    validation_indices = _split_indices(
        grouper,
        lambda user_ratings: user_ratings.iloc[int(user_ratings.shape[0] * 0.5):
                                               int(user_ratings.shape[0] * 0.75)].index.values)
    
    test_indices = _split_indices(
        grouper,
        lambda user_ratings: user_ratings.iloc[int(user_ratings.shape[0] * 0.75):].index.values)
    
    return train_indices, validation_indices, test_indices


train_indices, validation_indices, test_indices = split(path_data_orders)

# save results
with open(train_path, 'wb') as out:
    pickle.dump(train_indices, out)

with open(valid_path, 'wb') as out:
    pickle.dump(validation_indices, out)

with open(test_path, 'wb') as out:
    pickle.dump(test_indices, out)

### Файлы с индексами готовы - теперь скомпонуем и запишем в файл все необходимое

In [103]:
def read_and_concat_same_data(df_train_path, df_prior_path, df_orders_path):
    
    df_train = pd.read_csv(df_train_path)
    df_prior = pd.read_csv(df_prior_path)
    df_orders = pd.read_csv(df_orders_path)
    
    order_products_full = pd.concat([df_train, df_prior])
    
    return order_products_full, df_orders


def get_split_df(train_path, valid_path, test_path, df_orders, df_order_products_full):
    
    with open(train_path, 'rb') as input:
        train_indices = pickle.load(input)

    with open(valid_path, 'rb') as input:
        validation_indices = pickle.load(input)

    with open(test_path, 'rb') as input:
        test_indices = pickle.load(input)
        
    train_order_ids = df_orders.loc[train_indices]['order_id'].values
    validation_order_ids = df_orders.loc[validation_indices]['order_id'].values
    test_order_ids = df_orders.loc[test_indices]['order_id'].values
    
    train_df = df_order_products_full.loc[train_order_ids]
    validation_df = df_order_products_full.loc[validation_order_ids]
    test_df = df_order_products_full.loc[test_order_ids]
    
    return train_df, validation_df, test_df


# считается что уже имеем 3 файла с индексами
df_order_products_full, df_orders = read_and_concat_same_data(path_data_train, path_data_prior, path_data_orders)
train_df, validation_df, test_df = get_split_df(train_path, valid_path, test_path, df_orders, df_order_products_full)

# write new df's to file:
train_df.to_csv(df_train_path, index=False)
validation_df.to_csv(df_valid_path, index=False)
test_df.to_csv(df_test_path, index=False)

### notes


In [44]:
# Лучше хранить в словаре, так как могут быть разные n для разных юзеров (как переделать текущий дф):
# d = df.to_dict('index')
# d[user_num].values() - выдает словарь список - можно пройтись в цикле и переписать

# в классе заюзаем dict и просто впишем массив, подфиксить под это надо predict(), fit(), set_similarity_df()

# учесть то, что для разных юзеров разное n:
# добавть параметр if sim<min_sim: отбрасываем юзера из топа => в таком случае n будет меньше стандартного

# создать норм систему обработки ошибок, а то это никуда не годится...

In [ ]:
h = pd.DataFrame(columns=[i for i in range(1, 3)])
d = h.to_dict('index')
for num in d[3].values():
    print(num)

# Метрички
#### PS Подсчитаны для части пользователей, уж больно долго это все

In [6]:
# AP@k
def apk(actual, predicted, k=20):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

# MAP@K
def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

## Подсчет метрики для маленького сета dim = 10, top_n = 5
### По 30 продуктам и top_n = 5

In [4]:
# файл, где лежат результаты
path_to_res = 'data/results.csv'

# необходимые файлы: 
# 1. исходный из keggle orders
path_to_orders = 'data/orders.csv'

# 2. созданный-смерженный как надо методами из п.II
df_train_path = 'data_split/train_data.csv'

# размерность для декомпозиции
n_comp = 10

# создаем экземпляр класса с вышеописанными параметрами
Recommender = UserBasedRecommender(path_to_orders, df_train_path, n_comp)

# Загружаем уже посчитанные результаты
Recommender.set_similarity_df(path_to_res)

# Обучение включает с параметром loaded_df=True создаст только необходимые для пресказания матрицы, 
# не относящиеся к уже имеющимся результатам (по дефолту False и для крохотных размерностей считалось 3 часа...)
Recommender.fit(loaded_df=True)

df_valid_data = 'data_split/validation_data.csv'
Recommender.prepare_validation_data(df_valid_data)

In [29]:
actual, predicted = Recommender.check_metrics(count_users=5)
print(f'MAP@k dim=10, top=5, on 10 users and check top-5: {mapk(actual.values(), predicted.values(), k=30)}')

MAP@k dim=10, top=5, on 10 users and check top-5: 0.012


In [27]:
actual, predicted = Recommender.check_metrics(count_users=100)
print(f'MAP@k dim=10, top=5, on 100 users and check top-5: {mapk(actual.values(), predicted.values(), k=30)}')

MAP@k dim=10, top=5, on 100 users and check top-5: 0.04249942849142009


In [32]:
actual, predicted = Recommender.check_metrics(count_users=1000)
print(f'MAP@k dim=10, top=5, on 1000 users and check top-5: {mapk(actual.values(), predicted.values(), k=30)}')

D:\Soft\Anaconda\lib\site-packages\ipykernel_launcher.py:200: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


No user with id:  4
No user with id:  13
No user with id:  22
No user with id:  30
No user with id:  66
No user with id:  72
No user with id:  81
No user with id:  92
No user with id:  116
No user with id:  120
No user with id:  139
No user with id:  178
No user with id:  198
No user with id:  201
No user with id:  225
No user with id:  240
No user with id:  242
No user with id:  260
No user with id:  271
No user with id:  275
No user with id:  282
No user with id:  285
No user with id:  314
No user with id:  317
No user with id:  326
No user with id:  344
No user with id:  365
No user with id:  377
No user with id:  383
No user with id:  384
No user with id:  395
No user with id:  400
No user with id:  408
No user with id:  431
No user with id:  436
No user with id:  441
No user with id:  456
No user with id:  461
No user with id:  472
No user with id:  477
No user with id:  483
No user with id:  518
No user with id:  525
No user with id:  549
No user with id:  561
No user with id:  5

In [33]:
print(f'MAP@k dim=10, top=5, on 1000 users and check top-5: {mapk(actual.values(), predicted.values(), k=30)}')

MAP@k dim=10, top=5, on 1000 users and check top-5: 0.039625504302369975


## Подсчет метрики для большого сета dim = 50, top_n = 100
### По 30 продуктам top_n = (5, 10, 15, 100)

In [13]:
# файл, где лежат результаты
path_to_res = 'data/results_top_100_dim_50.csv'

# необходимые файлы: 
# 1. исходный из keggle orders
path_to_orders = 'data/orders.csv'

# 2. созданный-смерженный как надо методами из п.II
df_train_path = 'data_split/train_data.csv'

df_valid_data = 'data_split/validation_data.csv'

# размерность для декомпозиции
n_comp = 50

# создаем экземпляр класса с вышеописанными параметрами
Recommender_1 = UserBasedRecommender(path_to_orders, df_train_path, n_comp)

# Загружаем уже посчитанные результаты
Recommender_1.set_similarity_df(path_to_res)

# Обучение включает с параметром loaded_df=True создаст только необходимые для пресказания матрицы, 
# не относящиеся к уже имеющимся результатам (по дефолту False и для крохотных размерностей считалось 3 часа...)
Recommender_1.fit(loaded_df=True)

df_valid_data = 'data_split/validation_data.csv'
Recommender_1.prepare_validation_data(df_valid_data)


In [31]:
actual, predicted = Recommender_1.check_metrics(count_users=5)
print(f'MAP@k dim=50, top=100, on 5 users and check top-100: {mapk(actual.values(), predicted.values(), k=30)}')

MAP@k dim=50, top=100, on 5 users and check top-100: 0.02222222222222222


In [21]:
actual, predicted = Recommender_1.check_metrics(count_users=5, sim_users_count=10)
print(f'MAP@k dim=50, top=100, on 5 users and check top-10: {mapk(actual.values(), predicted.values(), k=30)}')

MAP@k dim=50, top=100, on 5 users and check top-10: 0.02222222222222222


In [ ]:
actual, predicted = Recommender_1.check_metrics(count_users=10, sim_users_count=5)
print(f'MAP@k dim=50, top=100, on 10 users and check top-5: {mapk(actual.values(), predicted.values(), k=30)}')

In [19]:
actual, predicted = Recommender_1.check_metrics(count_users=10, sim_users_count=10)
print(f'MAP@k dim=50, top=100, on 10 users and check top-10: {mapk(actual.values(), predicted.values(), k=30)}')

MAP@k dim=50, top=100, on 10 users and check top-10: 0.01912037037037037


In [23]:
actual, predicted = Recommender_1.check_metrics(count_users=10, sim_users_count=15)
print(f'MAP@k dim=50, top=100, on 10 users and check top-15: {mapk(actual.values(), predicted.values(), k=30)}')

MAP@k dim=50, top=100, on 10 users and check top-15: 0.01912037037037037


In [24]:
actual, predicted = Recommender_1.check_metrics(count_users=100, sim_users_count=5)
print(f'MAP@k dim=50, top=100, on 100 users and check top-5: {mapk(actual.values(), predicted.values(), k=30)}')

D:\Soft\Anaconda\lib\site-packages\ipykernel_launcher.py:217: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


No user with id:  4
No user with id:  13
No user with id:  22
No user with id:  30
No user with id:  66
No user with id:  72
No user with id:  81
No user with id:  92
No user with id:  116
No user with id:  120
MAP@k dim=50, top=100, on 100 users and check top-5: 0.05789145874244543


In [ ]:
# Рассматривать большое число соседей не имеет смысла, так как результат либо тот же либо сильно хуже
# 5-15 довольно оптимальные числа и по времени занимает меньше, чем предсказывать покупки по 100 схожим юзерам

# Размерность декомпозиции тоже влияет, с dim=50 результаты в целом получше, чем с 10, 
# но считалось это в разы дольше - может имеет смысл взять около 25